In [64]:
# import
import pandas as pd
from collections import OrderedDict
from operator import itemgetter    
import re
from bs4 import BeautifulSoup

In [65]:
df = pd.read_csv(r'../QBT/Empirical.csv', sep=',')
print(df.shape)


(21138, 23)


In [66]:
# Removing the "code" and "a" tag first, we want to delete what is inside the tag aswell, remvoing "a href" links and code snippets.

new_list = []
list_body = df["Body"].to_list()
temp = list_body

#print(list_body[2])
#print("-----")

for x in range(len(temp)):
    soup = BeautifulSoup(temp[x])
    codetags = soup.find_all(['code', 'a'])
    for codetag in codetags:
        codetag.extract()
        
    #print(soup)
    #print(str(soup))
    
    new_list.append(str(soup))

#print(len(new_list))
#print(new_list[2])
#print(df["Body"][1])

print("-----")
df["Body_new"] = new_list

#print(df["Body_new"][2])


-----


In [67]:
# Removing chacter and tags from the dataframe

# Removing the remaning tags
df["Body_new"] = df.Body_new.apply(lambda x: re.sub('<[^<]+?>', '', x))

# Removing ''
df["Body_new"] = df.Body_new.apply(lambda x: re.sub('\'', '', x))

# Removing ""
df["Body_new"] = df.Body_new.apply(lambda x: re.sub('\"', '', x))

df["Body_new"] = df.Body_new.apply(lambda x: re.sub('\.', ' ', x))

df["Body_new"] = df.Body_new.apply(lambda x: re.sub('\,', ' ', x))

df["Body_new"] = df.Body_new.apply(lambda x: re.sub('\!', ' ', x))

df["Body_new"] = df.Body_new.apply(lambda x: re.sub('\?', ' ', x))

df["Body_new"] = df.Body_new.apply(lambda x: re.sub('\\n', ' ', x))


print(df["Body_new"][1])

I have a node function that gets a link to YouTube video and sends a request to    Its a website that allows to download MP3 from YT  Then it parses the response HTML to get a direct download link from the document (using a library called cheerio  but if Youre not familiar with it  its just for scraping the link from HTML)  and afterwards opens the link to download the MP3  My code:  While working on my machine  this code works correctly  The console log gives a link which I can open on my browser and see that the mp3 is actually downloading  But when I run it on Lambda  the link I get has the correct format  but its simply not working  It redirects to some non-existing domain  Im not entirely sure how is this even possible  The only thing I might think about is that the website may think that the program is a bot (logically) and give a wrong link (which sounds pretty bizzare)  But I decided to also send some user-agent headers  It didnt work either  Im really confused about how can th

In [68]:
# Getting all posts with Empirical tags, this is beacuse our initial dataset contains posts with a "master"-tag of the set of tags

list_of_tages13 = ["<serverless>", "<faas>", "<serverless-framework>", "<aws-serverless>", "<openwhisk>", "<aws-lambda>", "<aws-sam>", "<aws-sam-cli>", 
"<serverless-architecture>", "<serverless-offline>", "<vercel>", "<serverless-plugins>", "<localstack>"]

What_we_want = df[df["Tags"].str.contains('|'.join(list_of_tages13))]
print(len(What_we_want))

df = What_we_want



20926


In [69]:
#  Code for creating dictonary that is counting the number of posts iwth the specific tag.
#What_we_want = What_we_want["Tags"].str.replace(r'<', '')
#What_we_want = What_we_want.str.replace(r'>', ' ')
#tags_list = What_we_want.tolist()

#list_of_tags = []
#for x in range(len(tags_list)):
#    list_of_tags.extend(tags_list[x].split())


#d = {i:list_of_tags.count(i) for i in list_of_tags}
#print(OrderedDict(sorted(d.items(), key = itemgetter(1), reverse = True)))

In [70]:
print(df.columns)

Index(['Id', 'PostTypeId', 'AcceptedAnswerId', 'ParentId', 'CreationDate',
       'DeletionDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId',
       'OwnerDisplayName', 'LastEditorUserId', 'LastEditorDisplayName',
       'LastEditDate', 'LastActivityDate', 'Title', 'Tags', 'AnswerCount',
       'CommentCount', 'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate',
       'ContentLicense', 'Body_new'],
      dtype='object')


In [71]:
synonyms = [" cost"," pricing"," Fee "," Charge","Expensive", " Pay "," Payment"," bill " ," Cheap" ," Budget"]
count = 0
for word in synonyms:
    print("----" + str(word) + "----")
    print("Body count: " + str(len(df[df['Body_new'].str.contains(word, case=False)])))
    count= count + len(df[df['Body_new'].str.contains(word, case=False)])
print(count)

---- cost----
Body count: 277
---- pricing----
Body count: 30
---- Fee ----
Body count: 4
---- Charge----
Body count: 65
----Expensive----
Body count: 71
---- Pay ----
Body count: 44
---- Payment----
Body count: 33
---- bill ----
Body count: 15
---- Cheap----
Body count: 31
---- Budget----
Body count: 6
576


In [73]:
# filtering out the final dataset

#df_temp = pd.DataFrame()
df_filtered = df[df['Body_new'].str.contains('|'.join(synonyms), case=False)]
print(len(df_filtered))

#for i in range(len(synonyms)-1):
#    temp = df[df['Body_new'].str.contains(synonyms[i+1], case=False)]
#    df_filtered = temp.merge(df_filtered, on='Id', how='outer')
    
print(len(df_filtered))
df_filtered.to_csv(r'../final_dataset_0401.csv', sep= ",", index = False)



499
499


In [28]:
pd.set_option('display.max_colwidth', None)
for word in synonyms:
    print("----" + word + "----")    
    temp = df[df['Body_new'].str.contains(word, case=False)]
    
    size = len(temp)
    
    if size <= 30:
        print(temp["Body_new"].sample(frac = 1))
    elif size > 30 and size <= 100:
        print(temp["Body_new"].sample(frac = 0.4))
    elif size > 100 and size <= 300:
        print(temp["Body_new"].sample(frac = 0.1))
    elif size > 300 and size <= 1000:
        print(temp["Body_new"].sample(frac = 0.05))
    else:
        print(temp["Body_new"].sample(frac = 0.02))


    
    


---- cost----
19828                                                                                                      Im evaluating some scenarios for my needs with media extraction from mp4 video files to mp3 using AWS lambda approach   The main requirement is once a new mp4 file is available or saved in some S3 bucket by a custom application the Lambda function will be triggered and send metadata to Dynamo DB and then the lambda function will extract the audio and store it in another S3 bucket  There are available four options as described below to design the Lambda function:  For example  use Java plus JAVE encoder library to do the job (with embed  ffmpeg binary in the lib/jar) Use NodeJS with some npm package with media encode capacity (including spawn ffmpeg process) Use NodeJS with AWS Elastic Transcode Service  Use Java with AWS Elastic Transcode Service  Which of option above will result in less cost in terms of resource (memory/cpu usage tiers)  I’m guessing the two initia

16657                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

16780                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [ ]:

#temp1 = df[df['Body'].str.contains("cost")]
print(len(temp1))
#temp2 = df[df['Tags'].str.contains("cost")]
#merged = temp1.merge(temp2, on='Post Link', how='outer')

#temp2 = df[df['Title'].str.contains("cost")]
#merged2 = merged.merge(temp2, on='Post Link', how='outer')

#temp2 = df[df['Body'].str.contains("price")]
#merged3 = merged2.merge(temp2, on='Post Link', how='outer')

#temp2 = df[df['Tags'].str.contains("price")]
#merged4 = merged3.merge(temp2, on='Post Link', how='outer')

#temp2 = df[df['Title'].str.contains("price")]
#merged5 = merged4.merge(temp2, on='Post Link', how='outer')

#temp2 = df[df['Body'].str.contains("pricing")]
#merged6 = merged5.merge(temp2, on='Post Link', how='outer')

#temp2 = df[df['Tags'].str.contains("pricing")]
#merged7 = merged6.merge(temp2, on='Post Link', how='outer')

#temp2 = df[df['Title'].str.contains("pricing")]
#merged8 = merged7.merge(temp2, on='Post Link', how='outer')

#temp2 = df[df['Body'].str.contains("pay")]
#merged9 = merged8.merge(temp2, on='Post Link', how='outer')

#temp2 = df[df['Tags'].str.contains("pay")]
#merged10 = merged9.merge(temp2, on='Post Link', how='outer')

#temp2 = df[df['Title'].str.contains("pay")]
#merged11 = merged10.merge(temp2, on='Post Link', how='outer')

temp1 = df[df['Body'].str.contains("fee")]

temp2 = df[df['Tags'].str.contains("fee")]
merged10 = temp1.merge(temp2, on='Post Link', how='outer')

temp2 = df[df['Title'].str.contains("fee")]
merged11 = merged10.merge(temp2, on='Post Link', how='outer')



print(len(merged11))
merged12 = merged11.drop_duplicates(subset=['Post Link'])
print(len(merged12))


In [ ]:
export = merged12["Post Link"]
print(export.head())
export = export.sample(frac=1).reset_index(drop=True)
export = export.iloc[:105,]
print(export.shape)
print(export.head())


In [ ]:
#export.to_csv(r'../exportFee.csv', sep= ",", index = False)